# lascar Example with CW Traces

Capturing is the same as usual. We'll capture a rather large amount of trace (3000) to show off the speed of lascar:

In [ ]:
%run "../Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "../Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "../Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 3000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

## Using lascar to Break Traces

The first thing we'll need to do is to put our traces into a format that lascar can understand. Luckily, lascar includes a trace container format that works with numpy arrays:

In [ ]:
from lascar import *
mycontainer =  TraceBatchContainer(trace_array, textin_array)

Next, we'll need to create a leakage function. This is a simple CPA attack, so we can just use lascar's sbox lookup and hamming function. We make a generator function here since it will allow us to easily create a new function for each subkey:

In [ ]:
from lascar.tools.aes import sbox

def generate_selection_function(byte):
    def selection_with_guess(value, guess): # selection_with_guess function must take 2 arguments: value and guess
        return hamming(sbox[value[byte] ^ guess])
    return selection_with_guess

Finally, we can create a list of `CpaEngine` to break each subkey:

In [ ]:
guess_range = range(256)

In [ ]:
cpa_engines = [CpaEngine("cpa_%02d" % i, generate_selection_function(i), guess_range) for i in range(16)]

And then actually run the attack:

In [ ]:
session = Session(mycontainer, engines=cpa_engines).run(batch_size=50)

lascar should break all of the traces rather quickly. All that's left is to interpret the results:

In [ ]:
for i in range(16):
    results = cpa_engines[i].finalize()
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))